In [1]:
from selenium import webdriver
import base64, pickle, time, os, pickle
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil

### 1. Get Youtube Images

##### (1) Get Youtube Driver & Save Element Screen

In [2]:
def getDriver(youtube_url):
    driver =  webdriver.Chrome()
#     driver.set_window_size(1920, 1080)
    driver.get(youtube_url)
    return driver

def saveScreen(driver, filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('screenshot.png')
    im = pil.open('screenshot.png')
    left = location['x']
    top = location['y'] * 2
    right = left + size['width'] * 2
    bottom = top + size['height'] * 2

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

##### (2) Check Element

In [3]:
def checkElement(driver, selector):
    result = True
    try:
        driver.find_element_by_css_selector(selector)
    except:
        result = False
    return result

##### (3) Skip Ad

In [4]:
skipAd_selector = ".videoAdUiSkipButton.videoAdUiAction.videoAdUiFixedPaddingSkipButton"
closeAd_selector = ".adDisplay.extra-padding .close-button"

def skipAd(driver):
    if checkElement(driver, skipAd_selector):
        driver.find_element_by_css_selector(skipAd_selector).click()
    
def closeAd(driver):
    if checkElement(driver, closeAd_selector):
        driver.find_element_by_css_selector(closeAd_selector).click()

##### (4) Make Directory

In [5]:
def makeDir(youtube_url):
    dirList = os.listdir("./data/images/")
    dirName = youtube_url.split("v=")[1]
    isDir = True
    if dirName not in dirList:
        os.makedirs("./data/images/" + dirName)
        isDir = False
    return isDir, dirName

#### (5) Save Images

In [6]:
def saveImages(youtube_url, start_term, image_count, save_term, dirName):
    
    # open driver
    driver = getDriver(youtube_url)

    # skip ad
    time.sleep(6)
    skipAd(driver)

    # save images
    time.sleep(start_term)
    for idx in range(1, image_count+1):
        saveScreen(driver, './data/images/' + dirName + '/screenshot' + str(idx) + '.png')
        time.sleep(save_term)
        # print('saved : screenshot' + str(idx) + '.png')
        
        # check end of video
        # if checkElement(driver, ".ytp-upnext-autoplay-icon"):
        #   break
        
    # close driver
    driver.close()

#### (6) make image list & thumnail

In [7]:
def makeTunmnail(image_count, dirName):
    
    # make image list
    filenames = []
    for idx in range(1, image_count + 1):
        filename = '/Users/rada/Documents/code/git/crawling/data/images/' + dirName  + '/screenshot' + str(idx) + '.png'
        filenames.append(filename)

    # make thumnail
    size = (512, 512)
    thumnails = []
    for filename in filenames:
        im = pil.open(filename)
        im.thumbnail(size)
        savename = filename.split("/")
        savename[len(savename)-1] = savename[len(savename)-1].replace(".png","_thumbnail.png")
        savename = "/".join(savename)
        im.save(savename, "png")
        thumnails.append(savename)
        
    return thumnails

### 2. Google Vision API

In [80]:
# single thread
def vision(filenames):
    url = 'https://cloud.google.com/vision/'

    # open site
    driver =  webdriver.Chrome()
    driver.get(url)

    # enter iframe
    iframe = driver.find_element_by_css_selector("#vision_demo_section iframe")
    driver.switch_to_frame(iframe)
    
    result = []
    
    for filename in filenames:
        try:
            # file upload
            driver.find_element_by_id("input").send_keys(filename)

            # check done analytics
            delay = 0
            for _ in range(10):
                if driver.find_element_by_css_selector("#results").text != '':
                    break
                time.sleep(1)
                delay += 1
            # print("Analytics Delay Time : {} sec".format(delay))

            if delay >= 10:
                continue

            # check face
            isFace = True
            try:
                driver.find_element_by_css_selector('[data-type="faceAnnotations"]')
            except:
                isFace = False

            face_data_list = []

            if isFace:
                face_list = driver.find_elements_by_css_selector('.face.style-scope.vs-faces')
                for idx, face in enumerate(face_list):
                    faceData = {
                        "name":"face " + str(idx+1),
                        "confidence": face.find_element_by_css_selector('.confidence .conf-score.style-scope.vs-faces').text
                    }          
                    expressions = face.find_elements_by_css_selector('.type-row.style-scope.vs-faces')
                    expressionData = {}
                    for expression in expressions:
                        name = expression.find_element_by_css_selector('#label').text
                        state = expression.find_element_by_css_selector('#text').text
                        if "Likely" in state or "Possible" in state:
                            state = 1
                        else:
                            state = 0
                        expressionData[name] = state
                    faceData["expressions"] = expressionData
                    face_data_list.append(faceData)  

            result.append(face_data_list)    

            # close image
            for _ in range(10):
                if checkElement(driver, "#exit"):
                    break
                else:
                    time.sleep(0.1)
            driver.find_element_by_css_selector("#exit").click()
        except:
            pass
#             print("anaytics error. (skip)")
    driver.close()
    return result

#### Save Images & Thumnails

In [72]:
# 여행에 미치다 (라오스)
# youtube_url = "https://www.youtube.com/watch?v=O32OrUMVhPo" 

# 패밀리맨
# youtube_url = "https://www.youtube.com/watch?v=rkEwymrfkEY" 

# 어바웃 타임
# youtube_url = "https://www.youtube.com/watch?v=tnyWkyDGWuM" 

# 악녀 제작보고회
youtube_url = "https://www.youtube.com/watch?v=CdAHvaCT4nI"

# 도깨비 
# youtube_url = "https://www.youtube.com/watch?v=2kZg7byEYGA"

In [62]:
%%time
isDir, dirName = makeDir(youtube_url)
start_term = 50 # 0초 후 부터 이미지 생성
image_count = 50 # 50개의 이미지 생성
save_term = 0.1 # 저장 텀
saveImages(youtube_url, start_term, image_count, save_term, dirName)

# make thumnail
thumnails = makeTunmnail(image_count, dirName)

CPU times: user 35.2 s, sys: 723 ms, total: 35.9 s
Wall time: 2min


In [74]:
thumnails = makeTunmnail(50, "CdAHvaCT4nI")

#### Run Vision API

In [81]:
%%time
result = vision(thumnails)

CPU times: user 31.7 ms, sys: 12.9 ms, total: 44.6 ms
Wall time: 10 s


In [64]:
expression_list = ['Joy', 'Sorrow', 'Anger', 'Surprise', 'Exposed', 'Blurred', 'Headwear']
person_count = 0
scene_count = 0

expressions = {}
for expression in expression_list:
    expressions[expression] = 0

for data in result:
    scene_count += 1
    person_count += len(data)
    for face in data:
        confidence = float(face["confidence"].replace("%", "")) * 0.01
        confidence = round(confidence, 2)
        for expression in expression_list:
            number = int(face['expressions'][expression]) * confidence / person_count
            expressions[expression] = round(expressions[expression] + number, 2)

In [67]:
summary_data = {
    "scene_count":scene_count,
    "person_count":person_count,
    "expressions":expressions,
}

# save
path = '/Users/rada/Documents/code/git/crawling/data/images/' + dirName  + '/result.plk'
pickle.dump(summary_data, open(path, 'wb'))

# read
with open(path, 'rb') as file:
    summary_data = pickle.load(file)
    print(summary_data)

{'scene_count': 37, 'person_count': 34, 'expressions': {'Joy': 0.06, 'Sorrow': 0.0, 'Anger': 0.0, 'Surprise': 0.0, 'Exposed': 0.0, 'Blurred': 0.0, 'Headwear': 0.0}}


In [71]:
# read
dirName = "tnyWkyDGWuM"
path = '/Users/rada/Documents/code/git/crawling/data/images/' + dirName  + '/result.plk'
with open(path, 'rb') as file:
    summary_data = pickle.load(file)
    print(summary_data)

{'scene_count': 44, 'person_count': 27, 'expressions': {'Joy': 0.55, 'Sorrow': 0.0, 'Anger': 0.0, 'Surprise': 0.04, 'Exposed': 0.0, 'Blurred': 0.0, 'Headwear': 0.29}}
